<a href="https://colab.research.google.com/github/Matgat12/controle/blob/master/Pizzeria_chatbot_avec_OpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Commencez par installer les paquets nécessaires d'OpenAI
# Pour installer des paquets dans Colab, comme si on était dans sa terminale, on utilise !
! pip install openai==0.28

In [ ]:
! pip show openai

In [ ]:
# Importez votre clé d'API OpenAI. Pour faire cet exercise, il vous faudra créer votre propre clé.
# Pour en savoir plus : https://platform.openai.com/docs/overview
import openai
openai.api_key = "votre cle"

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo-1106", temperature=0): # l'un des paramètres qu'on peut utiliser et la réponse en format Json, très pratique !
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature
        )
    return response.choices[0].message["content"]

In [ ]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))

    return pn.Column(*panels)

In [ ]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display

context = [ {'role':'system', 'content':"""
Tu es un chatbot, un service automatisé de collecte des commandes pour une pizzeria. \
Tu te présentes d'abord auprès du client, puis tu récupéres sa commande, \
puis tu demandes s'il s'agit d'une commande à emporter ou d'une livraison. \
Tu attends de récupérer la totalité de la commande, puis la valides auprès du client et tu vérifies si le client souhaite ajouter autre chose. \
S'il s'agit d'une livraison, tu  demandes une adresse. \
Enfin, tu encaisses le paiement.\
Assure-toi de clarifier toutes les options, extras et tailles afin de bien identifier l'élément dans le menu.\
Tu répondes toujours dans un style court et très convivial. \
Le menu comprend \
pizza au pepperoni 12,95, 10,00, 7,00 \
pizza au fromage 10,95, 9,25, 6,50 \
pizza aux aubergines 11,95, 9,75, 6,75 \
frites 4,50, 3,50\
salade grecque 7.25\
Garnitures : \
fromage supplémentaire 2,00, \
champignons 1,50 \
saucisse 3,00 \
bacon canadien 3.50\
Sauce IA 1,50\
poivrons 1,00 \
Boissons : \
coca 3,00, 2,00, 1,00 \
sprite 3,00, 2,00, 1,00 \
eau en bouteille 5,00 \
tous les prix sont en euros
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Bonjour", placeholder='Tapez votre texte ici')
button_conversation = pn.widgets.Button(name="Envoyer")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=1200),
)

dashboard

In [ ]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'crée un tableau en format json de la commande précédente et limite ta réponse à ce tableau. Ta réponse doit être en format json et ne doit pas contenir de texte avant ni après. Détaille le prix de chaque article\
 Les champs doivent être 1) pizza 2) taille 3) liste des garnitures 4) liste des boissons, inclure la taille 5) liste des accompagnements, inclure la taille 6) prix total 7) livraison'},
)

response = get_completion_from_messages(messages, temperature=0)
print(response)